In [ ]:
!pip install music21 pretty_midi keras-self-attention pydot graphviz

In [15]:
from music21 import *
import mido, glob, numpy, pathlib, os, pickle, random
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, BatchNormalization as BatchNorm
from keras.utils import np_utils, plot_model
from keras_self_attention import SeqSelfAttention
import re


def main():
    # Load the notes used to train the models
    with open('all_notes-Right.pkl', 'rb') as filepath:
        right_notes = pickle.load(filepath)
    
    with open('all_notes-Left.pkl', 'rb') as filepath:
        left_notes = pickle.load(filepath)

    # Get all pitch names for both hands
    right_pitchnames = sorted(set(item for item in right_notes))

    left_pitchnames = sorted(set(item for item in left_notes))

    # Get all vocab for both hands
    Rn_vocab = len(set(right_notes))

    Ln_vocab = len(set(left_notes))

    # Prepare and Create the note sequences and Models for each hand
    right_input, norm_right_input = prepare_Note_Sequences(right_notes, right_pitchnames, Rn_vocab)
    left_input, norm_left_input = prepare_Note_Sequences(left_notes, left_pitchnames, Ln_vocab)
    right_model = create_Hand_Model(norm_right_input, Rn_vocab, 'Right')
    left_model = create_Hand_Model(norm_left_input, Ln_vocab,'Left')

    # Allows for a visual representation of the models
    #plot_model(right_model, to_file='right_model.png', show_shapes=True, show_layer_names=True, rankdir='TB', dpi=96)
    #plot_model(left_model, to_file='left_model.png', show_shapes=True, show_layer_names=True, rankdir='TB', dpi=96)

    # Generate the notes for both hands
    generated_notes = generate_Two_Hands_Notes(right_model, left_model, right_input, left_input, right_pitchnames, left_pitchnames, Rn_vocab, Ln_vocab)
    right_prediction_output = []
    left_prediction_output = []

    # Seperate the generated notes from the outputted pair into their respective lists
    for pair in generated_notes:
        right_note, left_note = pair
        right_prediction_output.append(right_note)
        left_prediction_output.append(left_note)


    #print(right_prediction_output)
    #print(left_prediction_output)

    # Transpose the left hand notes into a pitch that matches the right hand's generated notes
    t_left_notes = transpose_Left_Hand_To_Right_Key(right_prediction_output, left_prediction_output)
    #print(t_left_notes)
    
    # Combine the generated and transposed notes into a single MIDI file
    create_Midi(right_prediction_output, t_left_notes)


"""
    Prepare the note sequences and corresponding network input for a model.

    Args:
        music_notes (list): List of notes or chords.
        unique_notes (list): List of unique pitch names.
        otal_unique_notes (int): Total number of unique pitches or chords.

    Returns:
        tuple: A tuple containing the network input sequences and normalized input.

    """
def prepare_Note_Sequences(music_notes, unique_notes, total_unique_notes):
    # Create a mapping between notes and integers and back
    note_to_int_mapping = {note: number for number, note in enumerate(unique_notes)}

    # Set the length of each input sequence
    sequence_length = 100

    # Generate the sequences for the network input and corresponding outputs
    input_sequences = [[note_to_int_mapping[note] for note in music_notes[i:i + sequence_length]] 
                     for i in range(len(music_notes) - sequence_length)]
                     
    # Reshape the network input into a format compatible with LSTM layers
    reshaped_input = numpy.reshape(input_sequences, (len(input_sequences), sequence_length, 1))

    # Normalize the input by dividing it by the total number of unique notes
    normalized_input = reshaped_input / float(total_unique_notes)

    return (input_sequences, normalized_input)

"""
    Create the structure of the neural network for generating hand movements.

    Args:
        network_input (ndarray): Input sequences for the network.
        n_vocab (int): Total number of unique pitches or chords.
        name (str): Name used for loading pre-trained weights.

    Returns:
        keras.models.Sequential: The compiled model for generating hand movements.
"""
def create_Hand_Model(network_input, n_vocab, name):
    # Create a sequential model
    model = Sequential()

    # Add the first LSTM layer with 512 units, recurrent dropout, and return sequences
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))

    # Add attention layer for focusing on important information
    model.add(SeqSelfAttention(attention_activation='sigmoid'))

    # Add the second LSTM layer with 512 units and return sequences
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3))

    # Add the third LSTM layer with 512 units
    model.add(LSTM(512))

    # Add batch normalization layer for stabilizing training
    model.add(BatchNorm())

    # Add dropout layer to prevent overfitting
    model.add(Dropout(0.3))

    # Add a dense layer with 256 units and ReLU activation
    model.add(Dense(256))
    model.add(Activation('relu'))

    # Add batch normalization layer
    model.add(BatchNorm())

    # Add dropout layer
    model.add(Dropout(0.3))

    # Add a dense layer with n_vocab units and softmax activation for output
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))

    # Compile the model with categorical cross-entropy loss and RMSprop optimizer
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load pre-trained weights
    model.load_weights(f'/content/drive/MyDrive/Notes_weights/new_weights-{name}.hdf5')

    return model

"""
    Generate a sequence of notes for both the right and left hand models.

    Args:
        right_model (keras.models.Sequential): Model for generating notes for the right hand.
        left_model (keras.models.Sequential): Model for generating notes for the left hand.
        right_input (ndarray): Input sequences for the right hand model.
        left_input (ndarray): Input sequences for the left hand model.
        right_pitchnames (list): List of unique pitch names for the right hand.
        left_pitchnames (list): List of unique pitch names for the left hand.
        right_n_vocab (int): Total number of unique pitches for the right hand.
        left_n_vocab (int): Total number of unique pitches for the left hand.

    Returns:
        list: A list of pairs of generated notes for the right and left hand.

"""
def generate_Two_Hands_Notes(right_model, left_model, right_input, left_input, right_pitchnames, left_pitchnames, right_n_vocab, left_n_vocab):
    # initialize starting points for both hands
    right_start = numpy.random.randint(0, len(right_input) - 1)
    left_start = numpy.random.randint(0, len(left_input) - 1)

    # Create mappings between integers and notes for both hands
    right_int_to_note = dict((number, note) for number, note in enumerate(right_pitchnames))
    left_int_to_note = dict((number, note) for number, note in enumerate(left_pitchnames))

    # Get the initial patterns for both hands
    right_pattern = right_input[right_start]
    left_pattern = left_input[left_start]

    # Generate notes
    prediction_output = []
    for note_index in range(200):
        # Prepare input data for both hands
        right_prediction_input = numpy.reshape(right_pattern, (1, len(right_pattern), 1))
        right_prediction_input = right_prediction_input / float(right_n_vocab)

        left_prediction_input = numpy.reshape(left_pattern, (1, len(left_pattern), 1))
        left_prediction_input = left_prediction_input / float(left_n_vocab)

        # Generate prediction for both hands
        right_prediction = right_model.predict(right_prediction_input, verbose=0)
        left_prediction = left_model.predict(left_prediction_input, verbose=0)

        # Get the index of the note with the highest probability for both hands
        right_index = numpy.argmax(right_prediction)
        left_index = numpy.argmax(left_prediction)

        # Get the actual note names for both hands
        right_result = right_int_to_note[right_index]
        left_result = left_int_to_note[left_index]

        # Append the generated notes to the output list
        prediction_output.append([right_result, left_result])

        # Update the patterns for both hands
        right_pattern.append(right_index)
        right_pattern = right_pattern[1:len(right_pattern)]

        left_pattern.append(left_index)
        left_pattern = left_pattern[1:len(left_pattern)]

    return prediction_output
   
"""
    Parse a key string into its tonic and mode components.

    Args:
        key_str (str): Key string representing the tonic and mode (e.g., 'C Major').

    Returns:
        tuple: A tuple containing the tonic and mode components.

"""
def parse_Key_String(key_str):
     # Use regular expressions to extract the tonic and mode from the key string
    tonic, mode = re.match(r'([A-Ga-g][#-]*)(.*)', key_str.strip()).groups()

    # Capitalize the first letter of the tonic and convert the mode to lowercase
    tonic = tonic[0].upper() + tonic[1:]
    mode = mode.strip().lower()

    return tonic, mode


"""
    Transpose the left-hand notes or chords to match the key of the right hand.

    Args:
        right_hand_notes (list): List of right hand notes or chords.
        left_hand_notes (list): List of left hand notes or chords.

    Returns:
        list: List of transposed left hand notes or chords.

"""
def transpose_Left_Hand_To_Right_Key(right_hand_notes, left_hand_notes):
    transposed_left_hand_notes = []

    for r_note, l_note in zip(right_hand_notes, left_hand_notes):
        r_tonic, r_mode = parse_Key_String(r_note[2])
        l_tonic, l_mode = parse_Key_String(l_note[2])

        r_key = key.Key(tonic=r_tonic, mode=r_mode)
        l_key = key.Key(tonic=l_tonic, mode=l_mode)

        # Calculate the interval between the right and left hand keys
        interval = r_key.tonic.pitchClass - l_key.tonic.pitchClass

        # Transpose the left-hand note or chord
        if len(l_note[0].split('.')) == 1:  # it's a note
            try:
                original_pitch = pitch.Pitch(l_note[0])
                transposed_pitch = original_pitch.transpose(interval)
                transposed_note = (str(transposed_pitch),) + l_note[1:]
            except pitch.PitchException:
                transposed_note = l_note
        else:  # it's a chord
            original_chord = [pitch.Pitch(int(pitch_class)) for pitch_class in l_note[0].split('.')]
            transposed_chord = [str(p.transpose(interval)) for p in original_chord]
            transposed_note = ('.'.join(transposed_chord),) + l_note[1:]

        transposed_left_hand_notes.append(transposed_note)

    return transposed_left_hand_notes

"""
    Create a MIDI file based on the generated prediction output for both hands.

    Args:
        prediction_output (list): List of generated notes or chords for the right hand.
        left_output (list): List of generated notes or chords for the left hand.

    Returns:
        None

"""  
def create_Midi(prediction_output, left_output):
    output_notes = []

    # set the initial tempo
    #number = random.randint(110,140)
    initial_tempo = tempo.MetronomeMark(number=120)
    initial_tempo.offset = 0.0
    output_notes.append(initial_tempo)

    # create note and chord objects based on the values generated by the model
    offset = 0.0
    for prediction, left in zip(prediction_output, left_output):
        pattern = prediction[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        left_pattern = left[0]
        # left hand pattern is a chord
        if ('.' in left_pattern) or left_pattern.isdigit():
            notes_in_chord = left_pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                try:
                    new_pitch = pitch.Pitch(current_note)  # Directly use the pitch.Pitch() function
                    new_note = note.Note()
                    new_note.pitch = new_pitch
                    new_note.storedInstrument = instrument.Piano()
                    notes.append(new_note)
                except pitch.PitchException:
                    print(f"Error with left hand chord note: {current_note}")
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # left hand pattern is a note
        else:
            try:
                new_note = note.Note(left_pattern)  # Directly use the note.Note() function
                new_note.offset = offset
                new_note.storedInstrument = instrument.Piano()
                output_notes.append(new_note)
            except pitch.PitchException:
                print(f"Error with left hand note: {left_pattern}")


        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')


if __name__ == '__main__':
    main()

Error with left hand chord note: 11
Error with left hand chord note: 11
Error with left hand chord note: 7
Error with left hand chord note: 9
Error with left hand chord note: 7
Error with left hand chord note: 2
Error with left hand chord note: 7
Error with left hand chord note: 11
Error with left hand chord note: 2
Error with left hand chord note: 4
Error with left hand chord note: 1
Error with left hand chord note: 11
Error with left hand chord note: 11
Error with left hand chord note: 7
Error with left hand chord note: 9
Error with left hand chord note: 7
Error with left hand chord note: 2
